# LSTM

## INTRO

Long Short-Term Memory (LSTM) is a type of recurrent neural network (RNN) architecture designed to handle sequence prediction problems. It is particularly effective in capturing long-term dependencies in sequential data.

- Sequential Input: LSTM processes input data sequentially, one element at a time, while maintaining an internal state.
- Forget Gate: At each time step, the LSTM decides what information to keep or forget from the internal state. This is done using a "forget gate" that takes the input and the previous state as inputs and outputs a number between 0 and 1 for each element in the internal state. A value of 1 means "keep this" and 0 means "forget this".
- Input Gate: Next, the LSTM decides what new information to store in the internal state. This is done through an "input gate" that takes the input and the previous state as inputs, processes them, and outputs a new candidate value for the internal state.
- Update the State: The internal state is updated by combining the information from the forget gate and the input gate. The forget gate decides what to remove from the state, and the input gate decides what to add.
- Output Gate: Finally, the LSTM decides what to output based on the updated internal state. This is done using an "output gate" that takes the input and the current state, processes them, and produces the output for the current time step.

# GRU

## Intro

Gated Recurrent Unit (GRU) is another type of recurrent neural network (RNN) architecture, similar to LSTM but somewhat simpler. It is designed to capture dependencies in sequential data.

- Update Gate: GRU has an update gate that controls how much of the previous state to keep and how much of the new state to add. It takes the input and the previous state, processes them, and decides what information to update in the current state.
- Reset Gate: There's also a reset gate that helps the model decide how much of the previous state to forget. It takes the input and the previous state, processes them, and decides what information to reset.
- Current Memory: GRU computes a new current memory based on the input, the previous state, and the update gate. It decides how much of the previous state to keep and how much of the new state to add.
- Output: Finally, GRU produces an output based on the current memory. This output can be used for predictions or passed to the next time step as input.


### Tensorflow

For the sake of my sanity, I am not building the models. Luckily, **Tensorflow.keras** comes with a semi-built-in option to construct RNNs with either/both methods (ore even some extra steps).

In [54]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

from data_funcs import *

In [55]:
portfolio_dict, portfolio = get_data()

100%|██████████| 31/31 [00:03<00:00,  7.94it/s]


In [56]:
portfolio['stocks']['AMD'].head()

,Adj Close,Close,High,Low,Open,Volume,day sin,returns
0,148.809998,148.809998,150.880005,148.740005,149.038605,2271179.0,-0.625923,-0.001543
1,147.755005,147.755005,149.020004,147.705505,148.800003,2178869.0,-0.642788,-0.007090
2,147.789993,147.789993,148.065399,147.250000,147.755005,1698140.0,-0.659346,0.000237
3,147.331802,147.331802,147.789993,147.119995,147.770004,1377528.0,-0.675590,-0.003100
4,147.470001,147.470001,147.559998,146.649994,147.339996,1586776.0,-0.691513,0.000938


The model will predict the necesary data to calculate the KPIs and performance meassures:

- High
- Low
- Close
- Adj Close

Using **all** of the variables in a 6-window periodn from that specific stock (df)

| $\hat{x}$<sub>t-5</sub> | $\hat{x}$<sub>t-4</sub> | $\hat{x}$<sub>t-3</sub> | $\hat{x}$<sub>t-2</sub> | $\hat{x}$<sub>t-1</sub> | $\hat{x}$<sub>t</sub> | $\hat{y}$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| df.iloc[ 0, : ] | df.iloc[ 1, : ] | df.iloc[ 2, : ] | df.iloc[ 3, : ] | df.iloc[ 4, : ] | df.iloc[ 5, : ] | [High<sub>6</sub>, Low<sub>6</sub>, Close<sub>6</sub>, Adj_Close<sub>6</sub>] |
| df.iloc[ 1, : ] | df.iloc[ 2, : ] | df.iloc[ 3, : ] | df.iloc[ 4, : ] | df.iloc[ 5, : ] | df.iloc[ 6, : ] | [High<sub>7</sub>, Low<sub>7</sub>, Close<sub>7</sub>, Adj_Close<sub>7</sub>] |
| df.iloc[ 2, : ] | df.iloc[ 3, : ] | df.iloc[ 4, : ] | df.iloc[ 5, : ] | df.iloc[ 6, : ] | df.iloc[ 7, : ] | [High<sub>8</sub>, Low<sub>8</sub>, Close<sub>8</sub>, Adj_Close<sub>8</sub>] |
| df.iloc[ 3, : ] | df.iloc[ 4, : ] | df.iloc[ 5, : ] | df.iloc[ 6, : ] | df.iloc[ 7, : ] | df.iloc[ 8, : ] | [High<sub>9</sub>, Low<sub>9</sub>, Close<sub>9</sub>, Adj_Close<sub>9</sub>] |
| ... | ... | ... | ... | ... | ... | ... |

## Split the data

In [57]:
df = portfolio['stocks']['AMD'].copy()
targets = df.loc[:, ['High', 'Low', 'Close', 'Adj Close']].to_numpy()
df = df.to_numpy()
X = []
Y = []
window = 6

for i in range(len(df)-window):
    r = [x for x in df[i:i+window]]
    X.append(r)
    Y.append(targets[i+window])

X = np.array(X)
Y = np.array(Y)

X.shape, Y.shape

((3089, 6, 8), (3089, 4))

In [58]:
idx = 8*int(X.shape[0]/10)
X_train = X[:idx, :, :]
Y_train = Y[:idx, :]
X_test =  X[idx:, :, :]
Y_test =  Y[idx:, :]

X_train.shape, Y_test.shape

((2464, 6, 8), (625, 4))

## Standarization

In [59]:
x_means = [np.mean(X_train[:, :, i]) for i in range(X_train.shape[2])]
x_stds = [np.std(X_train[:, :, i]) for i in range(X_train.shape[2])]

X_train_p = (X_train-x_means)/x_stds
X_test_p = (X_test-x_means)/x_stds
    
y_means = [np.mean(Y_train[:, i]) for i in range(Y_train.shape[1])]
y_stds = [np.std(Y_train[:, i]) for i in range(Y_train.shape[1])]

Y_train_p = (Y_train-y_means)/y_stds
Y_test_p = (Y_test-y_means)/y_stds

## Turn everything into a func

In [60]:
def preprocess(DF, window=6):
    df = DF.copy()
    targets = df.loc[:, ['High', 'Low', 'Close', 'Adj Close']].to_numpy()
    df = df.to_numpy()
    X = []
    Y = []

    for i in range(len(df)-window):
        r = [x for x in df[i:i+window]]
        X.append(r)
        Y.append(targets[i+window])

    X = np.array(X)
    Y = np.array(Y)
    
    idx = 7*int(X.shape[0]/10)
    idx2 = 9*int(X.shape[0]/10)
    X_train = X[:idx, :, :]
    Y_train = Y[:idx, :]
    X_test =  X[idx:idx2, :, :]
    Y_test =  Y[idx:idx2, :]
    X_val =  X[idx2:, :, :]
    Y_val =  Y[idx2:, :]
    
    x_means = [np.mean(X_train[:, :, i]) for i in range(X_train.shape[2])]
    x_stds = np.array([np.std(X_train[:, :, i]) for i in range(X_train.shape[2])])
    x_stds = np.where(x_stds == 0, 1, x_stds)

    X_train_p = (X_train-x_means)/x_stds
    X_test_p = (X_test-x_means)/x_stds
    X_val_p = (X_val-x_means)/x_stds
        
    y_means = [np.mean(Y_train[:, i]) for i in range(Y_train.shape[1])]
    y_stds = [np.std(Y_train[:, i]) for i in range(Y_train.shape[1])]

    Y_train_p = (Y_train-y_means)/y_stds
    Y_test_p = (Y_test-y_means)/y_stds
    Y_val_p = (Y_val-y_means)/y_stds
    
    return [X_train_p, X_test_p, X_val_p, x_means, x_stds], [Y_train_p, Y_test_p, Y_val_p, y_means, y_stds]

In [61]:
X, Y = {}, {}

for eq in portfolio_dict:
    for tckr in portfolio_dict[eq]:
        X[tckr], Y[tckr] = preprocess(portfolio[eq][tckr])

# Models

Because of the nature of the diversification, the project aims to buils a model for each type of equity. If computational load is not an issue, a model for every ticker would be better

In [62]:
models, checkpoints = {}, {}

for eq in portfolio_dict:
    for stck in portfolio_dict[eq]: 
        models[stck] = Sequential()
        models[stck].add(InputLayer((6, 8)))
        models[stck].add(LSTM(64))
        models[stck].add(Dense(8, 'relu'))
        models[stck].add(Dense(4, 'linear'))
        checkpoints[stck] = ModelCheckpoint('{}/'.format(stck), save_best_only=True)

In [63]:
models['VOO'].summary()

Model: "sequential_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_52 (LSTM)              (None, 64)                18688     
                                                                 
 dense_104 (Dense)           (None, 8)                 520       
                                                                 
 dense_105 (Dense)           (None, 4)                 36        
                                                                 
Total params: 19244 (75.17 KB)
Trainable params: 19244 (75.17 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [65]:
for eq in portfolio_dict:
        for stck in portfolio_dict[eq]:
                print(stck)
                models[stck].compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])
                models[stck].fit(X[stck][0], Y[stck][0], validation_data=(X[stck][1], Y[stck][1]), epochs=3, callbacks=[checkpoints[stck]])
                print("NEXT \n")

AMD
Epoch 1/3
61/68 [=========================>....] - ETA: 0s - loss: 0.2287 - root_mean_squared_error: 0.4783INFO:tensorflow:Assets written to: AMD\assets


INFO:tensorflow:Assets written to: AMD\assets


68/68 [==============================] - 6s 59ms/step - loss: 0.2094 - root_mean_squared_error: 0.4576 - val_loss: 0.4383 - val_root_mean_squared_error: 0.6620
Epoch 2/3
60/68 [=========================>....] - ETA: 0s - loss: 0.0109 - root_mean_squared_error: 0.1046INFO:tensorflow:Assets written to: AMD\assets


INFO:tensorflow:Assets written to: AMD\assets


68/68 [==============================] - 4s 55ms/step - loss: 0.0102 - root_mean_squared_error: 0.1010 - val_loss: 0.2511 - val_root_mean_squared_error: 0.5011
Epoch 3/3
59/68 [=========================>....] - ETA: 0s - loss: 0.0053 - root_mean_squared_error: 0.0725INFO:tensorflow:Assets written to: AMD\assets


...

175/175 [==============================] - 11s 44ms/step - loss: 0.3566 - root_mean_squared_error: 0.5972 - val_loss: 2.5323 - val_root_mean_squared_error: 1.5913
Epoch 2/3
173/175 [============================>.] - ETA: 0s - loss: 0.0680 - root_mean_squared_error: 0.2607INFO:tensorflow:Assets written to: ZC=F\assets


INFO:tensorflow:Assets written to: ZC=F\assets


175/175 [==============================] - 7s 40ms/step - loss: 0.0677 - root_mean_squared_error: 0.2602 - val_loss: 1.0630 - val_root_mean_squared_error: 1.0310
Epoch 3/3
174/175 [============================>.] - ETA: 0s - loss: 0.0452 - root_mean_squared_error: 0.2127INFO:tensorflow:Assets written to: ZC=F\assets


INFO:tensorflow:Assets written to: ZC=F\assets


175/175 [==============================] - 7s 38ms/step - loss: 0.0452 - root_mean_squared_error: 0.2126 - val_loss: 0.7423 - val_root_mean_squared_error: 0.8615
NEXT 

ZM=F
Epoch 1/3
175/175 [==============================] - 5s 11ms/step - loss: 0.0053 - root_mean_squared_error: 0.0727 - val_loss: 0.0131 - val_root_mean_squared_error: 0.1146
Epoch 2/3
175/175 [==============================] - 1s 6ms/step - loss: 0.0051 - root_mean_squared_error: 0.0717 - val_loss: 0.0167 - val_root_mean_squared_error: 0.1293
Epoch 3/3
175/175 [==============================] - 1s 7ms/step - loss: 0.0055 - root_mean_squared_error: 0.0740 - val_loss: 0.0133 - val_root_mean_squared_error: 0.1152
NEXT 

